In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from collections import defaultdict

In [ ]:
dataset_dir = '/ML/datasets/vqa-v1/'
ques_dir = dataset_dir+'questions/'
ans_dir = dataset_dir+'answers/'

ques_train_jsonpath = ques_dir + 'OpenEnded_mscoco_train2014_questions.json'
ques_val_jsonpath = ques_dir + 'OpenEnded_mscoco_val2014_questions.json'

ans_train_jsonpath = ans_dir + 'mscoco_train2014_annotations.json'
ans_val_jsonpath = ans_dir + 'mscoco_val2014_annotations.json'

In [ ]:
ques_train = json.load(open(ques_train_jsonpath, 'r'))
ques_val = json.load(open(ques_val_jsonpath, 'r'))

ans_train = json.load(open(ans_train_jsonpath, 'r'))
ans_val = json.load(open(ans_val_jsonpath, 'r'))

In [ ]:
print("Total no. of train questions are",len(ques_train['questions']))
print("Total no. of validation questions are",len(ques_val['questions']))

print("Total no. of train answers are",len(ans_train['annotations']))
print("Total no. of validation answers are",len(ans_val['annotations']))

In [ ]:
top3k = []

def process(subtype):
    
    ans = []
    ques = []
    
    if subtype == "train":
        ans = ans_train
        ques = ques_train
    else:
        ans = ans_val
        ques = ques_val
    
    data=[]
    annotations = ans['annotations']
    questions = ques['questions']
    imdir='COCO_%s2014_%012d.jpg'
    
    for i in tqdm(range(len(annotations))):
        annotation = annotations[i]
        question = questions[i]
        
        answ = annotation['multiple_choice_answer']
        im_path = imdir%(subtype, annotation['image_id'])
        ques = question['question']
        
        data.append({'im_path':im_path,'ques':ques,'answ':answ})
    
    ans_count = defaultdict(int)
    for d in data:
        a = d['answ']
        ans_count[a]+=1;
   
    global top3k
    print(top3k[:100])
    if(subtype=='train'):
        ans_count = dict(sorted(ans_count.items(), key=lambda item: item[1], reverse=True))
        top3k = [k for k,v in ans_count.items()][:3000]
        #print(top3k)
        #print(len(top3k))
    
    df = pd.DataFrame(data)
    #print(df)


    filtered_df = df.loc[df['answ'].isin(top3k)]
    #print(filtered_df)
    print('samples filtered out:', len(df)-len(filtered_df))
    
    rand_df = filtered_df.sample(frac=1).reset_index(drop=True)
    #print(rand_df)
    
    rand_df.to_csv(dataset_dir+subtype+'_data.csv', index=False)

In [ ]:
process("train")
process("val")

In [ ]:
train_data = pd.read_csv(dataset_dir+'train_data.csv')
train_data

In [ ]:
val_data = pd.read_csv(dataset_dir+'val_data.csv')
val_data

In [ ]:
train_data['answ'].value_counts